I'm gonna overwrite a lot of this notebook's old content. I changed the way I'm calculating wt, and wanna test that my training worked. 

In [28]:
from pearce.emulator import *
from pearce.mocks import cat_dict
import numpy as np
from os import path

In [29]:
import matplotlib
#matplotlib.use('Agg')
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [73]:
from GPy.models import GPKroneckerGaussianRegression, GPRegression
from GPy.kern import *
import h5py

In [31]:
training_file = '/home/users/swmclau2/scratch/xi_gg_zheng07_cosmo_v3/PearceXiggCosmo.hdf5'
test_file = '/home/users/swmclau2/scratch/xi_gg_zheng07_cosmo_test_v3/PearceXiggCosmoTest.hdf5'

In [32]:
f = h5py.File(training_file, 'r')

In [33]:
Ys = []
rbin = 1
for i in xrange(40):
    Ys.append(f['cosmo_no_%02d'%i]['a_1.000']['obs'].value[:, rbin])

In [34]:
n_hods = 100
start_idx = 0
X1 = f.attrs['cosmo_param_vals']
X2 = f.attrs['hod_param_vals'][start_idx:start_idx+n_hods]
Y = np.vstack(Ys)[:, start_idx:start_idx+n_hods]

In [35]:
X1.shape, X2.shape, Y.shape

((40, 7), (100, 4), (40, 100))

In [36]:
# how to add training errors?

In [72]:
c = np.eye(X1.shape[0])
print c.shape
print c.diagonal()

(40, 40)
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.]


In [67]:
K1 = RBF(input_dim = 7, ARD = False)+ Fixed(input_dim = 7, covariance_matrix=np.eye(X1.shape[0]))#+Bias(input_dim=7) + Linear(input_dim = 7, ARD = True)# + Bias(input_dim=7)# + White(input_dim=7)
K2 =  RBF(input_dim=4, ARD = False)#+ Linear(input_dim = 4, ARD = False) + Bias(input_dim=4)# + White(input_dim=4)

In [56]:
model = GPKroneckerGaussianRegression(X1, X2, Y,K1, K2 )

In [74]:
model = GPRegression(X1, Y, K1)

In [75]:
model.predict(testX1)

ValueError: operands could not be broadcast together with shapes (35,) (40,) 

In [57]:
model.optimize_restarts(num_restarts=10)

Optimization restart 1/10, f = -14033.5781655
Optimization restart 2/10, f = -14033.5781425
Optimization restart 3/10, f = -14033.5781817
Optimization restart 4/10, f = -14033.5781829
Optimization restart 5/10, f = -14033.578183
Optimization restart 6/10, f = -14033.578183
Optimization restart 7/10, f = -14033.5781827
Optimization restart 8/10, f = -14033.578183
Optimization restart 9/10, f = -14033.5781829
Optimization restart 10/10, f = -14033.5781255


In [58]:
K1.param_array

array([  3.10977459,  23.99289946,   0.02513485])

In [59]:
K2.param_array

array([ 0.66805255,  0.81678789])

In [60]:
f2 = h5py.File(test_file, 'r')

In [61]:
Y2s = []
for i in xrange(35):
    Y2s.append(f2['cosmo_no_%02d'%i]['a_1.000']['obs'].value[:, rbin])

In [62]:
testX1 = f2.attrs['cosmo_param_vals']
testX2 = f2.attrs['hod_param_vals']
testY = np.vstack(Y2s)

In [63]:
predY, _ = model.predict(testX1, testX2)

ValueError: operands could not be broadcast together with shapes (35,) (40,) 

In [ ]:
np.median( np.abs( (10**predY[:,0] - 10**testY.flatten(order='F'))/10**testY.flatten(order='F') )  ) 

In [ ]:
np.mean( np.abs( (10**predY[:,0] - 10**testY.flatten(order='F'))/10**testY.flatten(order='F') )  ) 

In [ ]:
predY[:,0]

In [ ]:
testY.flatten(order='F')

In [ ]:
k_dict = K1.to_dict()

In [ ]:
k_dict['parts'].keys()